# Chapter 10: Programming Guidelines in Julia
This notebook contains the sample source code explained in the book *Hands-On Julia Programming, Sambit Kumar Dash, 2021, bpb Publications. All Rights Reserved*.

In [1]:
using Pkg
pkg"activate ."
pkg"instantiate"

  Activating environment at `~/work/books/HOPJ/Chapter-10/Project.toml`


## 10.1 Introduction

## 10.2 Guidelines for Maintainability

**Use Functions**

**Code must be generalized and not type specific**

In [2]:
addone(x) = x + one(x)

addone (generic function with 1 method)

In [3]:
addone(5), addone(5.0), addone(5f0)

(6, 6.0, 6.0f0)

**Keep function scope narrow if that is the intent**

In [4]:
calc1() = 2, 1
calc2() = 1.1, 1.2

myvalidate(x::Int, y::Int) = x == y

myvalidate(calc1()...)

false

In [5]:
myvalidate(calc2()...)

LoadError: MethodError: no method matching myvalidate(::Float64, ::Float64)

In [6]:
function myvalidate2(x, y)
    xi, yi = Int(x), Int(y)
    return xi == yi
end

myvalidate2(2, 3.0)

false

In [7]:
myvalidate(2, Int(3.0))

false

**Append exclamation mark (!) to function that change parameters**

In [8]:
function double!(v::Vector)
    for i=firstindex(v):lastindex(v)
        v[i] *= 2
    end
    return v
end

double!([1,2,3])

3-element Vector{Int64}:
 2
 4
 6

**Avoid complex data types**

**Use function arguments matching to Base**

In [9]:
mymap(f, arr) = f.(arr)

mymap([1, 2, 3]) do x
    2x
end

3-element Vector{Int64}:
 2
 4
 6

 **Judicious usage of static parameters**

In [10]:
double(x::T) where {T<:Real} = 2x

double (generic function with 1 method)

In [11]:
double(x::Real) = 2x

double (generic function with 1 method)

In [12]:
methods(double)

# 1 method for generic function "double":
[1] double(x::Real) in Main at In[11]:1

In [13]:
myeltype(v::Vector{T}) where {T <: Real} = T

myeltype (generic function with 1 method)

In [14]:
myeltype([1.0, 2, 3])

Float64

**Do not overload Base methods for well defined data types**

In [15]:
module MyModule

Base.:(*)(x::Symbol, y::Symbol) = Symbol(string(x)*string(y))

end


Main.MyModule

In [16]:
:a*:b

:ab

In [17]:
a = i > 0 ? 1 : -1

LoadError: UndefVarError: i not defined

**Use try…catch only when absolutely needed**

### Coding Style

**Indentation**

```
for i = 1:200
    if a != 0
        if b != 0 
            # Do something
        end
    end
end
```

vs. 

```
for i = 1:200
    (a == 0 || b == 0) && continue
    # Do something 
end
```

**Use of Brackets**

```
if a == 0 && b == 0
    # Do something
end
```
is preferred over

```
if (a == 0) && (b == 0)
    # Do something
end
```

**Whitespaces around operators**

`f(arg1, key=v+1)` is preferred over `f(arg1, key = v + 1)`

**Define multiline functions with the verbose syntax**

The code is correct but the style is not preferred. 

```
F(x) = begin
    if iseven(x)
        2x
    else
        3x
    end
end
```

The following styles are better. 

```
function F(x) 
    if iseven(x)
        return 2x
    else
        return 3x
    end
end
```

or

```
F(x) = iseven(x) ? 2x : 3x
```

The less verbose code is normally given preference. 

**Character length limits of a line**





## 10.3 Functional Programming Principles

### Declarative Programming

While both pieces of code achieve the same functionality the second one is cleaner and is more like a declarative style of programming. 

In [18]:
a = [1, 2, 3]
for i=firstindex(a):lastindex(a)
    println(a[i])
end

1
2
3


In [19]:
foreach(println, a)

1
2
3


### Functions with no side-effects

The method `pop!` below modifies the argument, hence is a function with side effects. In Julia, a function that modifies the argument's state is typically specified with an exclamation (!) character at the end. Sometimes such methods help in better memory allocations and better optimized; hence preferred. 

In [20]:
a = [1, 2, 3]
pop!(a)
a

2-element Vector{Int64}:
 1
 2

### Higher Order Functions

Ways to define function of a function. 

In [21]:
f(x) = 2x
g(x) = x*x
h = g ∘ f
h(3)
36


36

### Immutability

While functions without side effects are easier to debug, knowing some data structures are not mutable ensures they can be packed into a memory layout that is easier to manage. `struct` and `mutable struct` help you achieve the same. 

## 10.4 Commonly Used Patterns

### Interfaces

Interfaces in Julia do not have to be named. A function definition can act as an interface. 

The module `geom` has two methods, `area` and `bbox` for geometric shapes. 

In [22]:
module geom
# All shape objects must have implementation for the following methods.
# area(x): shall return the area of a 2-D shape
function area end
# bbox(x): shall return the bounding box of a 2-D shape
function bbox end
end 

Main.geom

A **hard interface** throws an error if not implemented. 

In [23]:
module geom
area(x) = throw(Base.MethodError(area, (x,)))
bbox(x) = throw(Base.MethodError(bbox, (x,)))

struct Rectangle
   x; y; w; h
end
export Rectangle, area, bbox
end
using .geom
r = Rectangle(0, 0, 10, 20)

Rectangle(0, 0, 10, 20)

In [24]:
area(r)

LoadError: MethodError: no method matching area(::Rectangle)
[0mClosest candidates are:
[0m  area(::Any) at In[23]:2

A **soft interface** that just passes the input through. You could return `nothing` as well. 

In [25]:
module geom1
area(x) = identity(x)
bbox(x) = identity(x)

struct Rectangle
   x; y; w; h
end
export Rectangle, area, bbox
end
r = geom1.Rectangle(0, 0, 10, 20)

Main.geom1.Rectangle(0, 0, 10, 20)

In [26]:
geom1.area(r)

Main.geom1.Rectangle(0, 0, 10, 20)

### SuperTypes

Trying to abstract all the interfaces to a type `Shape`

In [27]:
module geom2
abstract type Shape end
area(x::Shape) = nothing
bbox(x::Shape) = nothing

struct Rectangle <: Shape
   x; y; w; h
end
end
r = geom2.Rectangle(0, 0, 10, 20)
Main.geom2.Rectangle(0, 0, 10, 20)

Main.geom2.Rectangle(0, 0, 10, 20)

In [28]:
geom2.area(r)

Let's add a `Polygon` type.

```
module geom2
…
struct Polygon <: Shape
    pts::Vector{Tuple{Any, Any}}
end
…
end
```

Adding a `Polygon` was easy but adding a `PolyLine` introduce additional complexities. `PolyLine` being an `OpenShape` does not have `area` while `Polygon` is a `ClosedShape` and has `area` method. 

In [29]:
module geom3
abstract type Shape end
bbox(x::Shape) = nothing

abstract type OpenShape end

struct PolyLine <: OpenShape
    pts::Vector{Tuple{Any, Any}}
end

abstract type ClosedShape end
area(x::ClosedShape) = nothing

struct Rectangle <: ClosedShape
   x; y; w; h
end

struct Polygon <: ClosedShape
    pts::Vector{Tuple{Any, Any}}
end

end

Main.geom3

### Holy Traits

With `Traits` or `Holy Traits` the same can be achieved easily. `HasAreaTrait` type and method combination provide a unique design pattern. 

In [30]:
module geom4
abstract type Shape end
bbox(x::Shape) = nothing

struct PolyLine <: Shape
    pts::Vector{Tuple{Any, Any}}
end

struct Rectangle <: Shape
   x; y; w; h
end

struct Polygon <: Shape
    pts::Vector{Tuple{Any, Any}}
end

abstract type HasAreaTrait end
struct HasArea <: HasAreaTrait end
struct HasNoArea <: HasAreaTrait end

HasAreaTrait(::Type) = HasNoArea()
HasAreaTrait(::Type{Polygon}) = HasArea()
HasAreaTrait(::Type{Rectangle}) = HasArea()

area(x::T) where {T<:Shape} = area(HasAreaTrait(T), x)
area(::HasArea, x::T) where {T <: Shape} = error("area(::HasArea, $T) method must be implemented")
area(::HasArea, r::Rectangle) = r.w*r.h

end

r = geom4.Rectangle(0, 0, 10, 20)
pg = geom4.Polygon([(0, 0), (10, 0), (10, 20), (0, 20)])
pl = geom4.PolyLine([(0, 0), (10, 0), (10, 20), (0, 20)])

Main.geom4.PolyLine(Tuple{Any, Any}[(0, 0), (10, 0), (10, 20), (0, 20)])

In [31]:
geom4.area(geom4.Rectangle(0, 0, 10, 20))

200

In [32]:
geom4.area(geom4.Polygon([(0, 0), (10, 0), (10, 20), (0, 20)]))

LoadError: area(::HasArea, Main.geom4.Polygon) method must be implemented

In [33]:
geom4.area(geom4.PolyLine([(0, 0), (10, 0), (10, 20), (0, 20)]))

LoadError: MethodError: no method matching area(::Main.geom4.HasNoArea, ::Main.geom4.PolyLine)
[0mClosest candidates are:
[0m  area([91m::Main.geom4.HasArea[39m, ::T) where T<:Main.geom4.Shape at In[30]:26

### Delegation

Writing the same code again and again can be cumbersome. `Square` type here is aggregating a `Rectangle` object and delegating the methods to the already implemented method of the `Rectangle` type. 

In [34]:
module geom4
abstract type Shape end
bbox(x::Shape) = nothing

struct Rectangle <: Shape
   x; y; w; h
end

bbox(x::Rectangle) = x

### Square has Rectangle as an attribute ###

struct Square <: Shape
    r::Rectangle
    Square(x, y, w) = new(Rectangle(x, y, w, w))
end

abstract type HasAreaTrait end
struct HasArea <: HasAreaTrait end
struct HasNoArea <: HasAreaTrait end

HasAreaTrait(::Type) = HasNoArea()
HasAreaTrait(::Type{Rectangle}) = HasArea()

area(x::T) where {T<:Shape} = area(HasAreaTrait(T), x)
area(::HasArea, x::T) where {T <: Shape} = error("area(::HasArea, $T) method must be implemented")
area(::HasArea, r::Rectangle) = r.w*r.h

### Code for delegation ###

for op in [:bbox, :area]
    @eval $op(s::Square) = ($op)(s.r)
end

end

Main.geom4

In [35]:
s = geom4.Square(0, 0, 10)

Main.geom4.Square(Main.geom4.Rectangle(0, 0, 10, 10))

In [36]:
geom4.area(s)

100

In [37]:
geom4.bbox(s)

Main.geom4.Rectangle(0, 0, 10, 10)

### Accessors

Julia composite types do not have any data hiding available natively. While in most cases data accessors are not needed. One may use them where there is a need to validate the values before updating the `mutable struct`. The width attribute in a `Rectangle` should not be negative. A variable naming beginning with an underscore (`_`) is kind of a directive that the variable is private. 

In [38]:
module geom
mutable struct Rectangle
    _x; _y; _w; _h
    function Rectangle(x, y, w, h)
        (w <= 0 || h <= 0) && error("width and height must be positive")
        new(x, y, w, h)
    end
end
w(r::Rectangle) = r._w
function w!(r::Rectangle, tw) 
    tw <= 0 && error("width cannot be negative")
    r._w = tw 
end
end


Main.geom

In [39]:
r = geom.Rectangle(0, 0, 10, 20)
geom.w!(r, 40)

40

In [40]:
geom.w(r)

40

## 10.5 Conclusion

## Exercises